In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
     .master("local[*]") \
     .appName("groupby_spark") \
     .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/05 10:45:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet("data/pq/green/*/*")

In [4]:
df_green.registerTempTable("green")

/home/xiangivyli/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [19]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc("hour", lpep_pickup_datetime) AS hour, 

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM 
    green
WHERE
    lpep_pickup_datetime >= "2020-01-01 00:00:00"
GROUP BY 
    zone, hour
""")

In [25]:
df_green_revenue \
    .repartition(6) \
    .write \
    .parquet("data/report/revenue/green", mode="overwrite")

In [21]:
df_green_revenue = 

+----+-------------------+------------------+--------------+
|zone|               hour|            amount|number_records|
+----+-------------------+------------------+--------------+
| 265|2020-01-07 08:00:00|             60.22|             2|
| 218|2020-01-02 11:00:00|115.46000000000001|             4|
|  74|2020-01-19 19:00:00| 749.4399999999996|            56|
| 196|2020-01-09 11:00:00|148.79000000000002|             9|
|  95|2020-01-04 12:00:00|224.34000000000012|            15|
|  64|2020-01-12 10:00:00|             45.04|             2|
|  11|2020-01-25 14:00:00|             55.51|             1|
| 129|2020-01-21 15:00:00|            139.54|            11|
|  52|2020-01-01 23:00:00|               6.3|             1|
|  82|2020-01-30 05:00:00|152.35000000000002|             8|
| 129|2020-01-15 22:00:00|272.95000000000005|            14|
| 145|2020-01-25 10:00:00|             80.22|             7|
| 260|2020-01-10 23:00:00| 248.9300000000001|            20|
|  65|2020-01-22 11:00:0

In [22]:
df_yellow = spark.read.parquet("data/pq/yellow/*/*")
df_yellow.registerTempTable("yellow")
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc("hour", tpep_pickup_datetime) AS hour, 

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM 
    yellow
WHERE
    tpep_pickup_datetime >= "2020-01-01 00:00:00"
GROUP BY 
    zone, hour
""")

In [27]:
df_yellow_revenue \
    .repartition(6) \
    .write \
    .parquet("data/report/revenue/yellow", mode="overwrite")

In [2]:
df_green_revenue = spark.read.parquet("data/report/revenue/green")
df_yellow_revenue = spark.read.parquet("data/report/revenue/yellow")

In [3]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed("amount", "green_amount") \
    .withColumnRenamed("number_records", "green_number_records")

In [4]:
df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed("amount", "yellow_amount") \
    .withColumnRenamed("number_records", "yellow_number_records")

In [5]:
df_join = df_green_revenue_tmp \
    .join(df_yellow_revenue_tmp, on=["hour", "zone"], how="outer")

In [6]:
df_join.write.parquet("data/report/revenue/total")

In [7]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  13|              null|                null|1214.8000000000002|                   56|
|2020-01-01 00:00:00|  18|               7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:00|  29|  

merge zone name into one table

In [10]:
df_zones = spark.read.parquet("data/zones/")

In [11]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [12]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [17]:
df_result.drop("LocationID", "zone").write.parquet("data/tmp/revenue-zones")